# Imports

In [ ]:
import requests
import scipy
import math
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sb
from sklearn.metrics import r2_score
from scipy.stats import pearsonr, norm
import matplotlib.pyplot as plt 
sb.set() 
from math import sqrt
from tensorflow import keras
from statsmodels.tools import categorical
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from datetime import datetime, timedelta
from keras.models import load_model
from joblib import dump, load

# Select variable to predict 
`String prediction_variable`

In [ ]:
# Main variable = 'MainIncoming_kW'
prediction_variable = 'MainIncoming_kW'

# Remove Units

In [ ]:
data = pd.read_csv('Training_Data.csv')
# remove units from length column and convert to float
data['Timestamp'] = data['Timestamp'].str.replace(' Singapore', '').astype(str)
data['MainIncoming_kW'] = data['MainIncoming_kW'].str.replace('kW', '').astype(float)
data['CHWRT'] = data['CHWRT'].str.replace('°C', '').astype(float)
data['CWRT'] = data['CWRT'].str.replace('°C', '').astype(float)
data['CHWST'] = data['CHWST'].str.replace('°C', '').astype(float)
data['CWST'] = data['CWST'].str.replace('°C', '').astype(float)
data['CHW Flow'] = data['CHW Flow'].str.replace('L/s', '').astype(float)
data['CDW Flow'] = data['CDW Flow'].str.replace('L/s', '').astype(float)
data['heat_in'] = data['heat_in'].str.replace('_RT', '').astype(float)
data['heat_out'] = data['heat_out'].str.replace('_RT', '').astype(float)
data['lift'] = data['lift'].str.replace('Δ°C', '').astype(float)
data['chwdt'] = data['chwdt'].str.replace('Δ°C', '').astype(float)
data['cwdt'] = data['cwdt'].str.replace('Δ°C', '').astype(float)
data['DemoSite Demo avgHum'] = data['DemoSite Demo avgHum'].str.replace('%RH', '').astype(float)
data['DemoSite Demo avgTemp'] = data['DemoSite Demo avgTemp'].str.replace('°C', '').astype(float)
data['DemoSite Demo Calculated Wet Bulb'] = data['DemoSite Demo Calculated Wet Bulb'].str.replace('°C', '').astype(float)
data

## Check data type (float64)

In [ ]:
print(data.dtypes)

# Extract Individual Dates

In [ ]:
data['Timestamp'] =pd.to_datetime(data['Timestamp'], format="%Y-%m-%dT%H:%M:%S%z")
# convert the timestamp to the corresponding day of the week
data['Day of Week'] = data['Timestamp'].dt.strftime('%A')
# Convert datetime to weekday numbers (Monday=1, Tuesday=2, etc.)
data['DayNumber'] = data['Timestamp'].dt.weekday
# add 1 to get days starting from 1
data['DayNumber'] = data['DayNumber'] + 1

data['Year'] = data['Timestamp'].dt.year
data['Month'] = data['Timestamp'].dt.month
data['Day'] = data['Timestamp'].dt.day
data['Hour'] = data['Timestamp'].dt.hour
data['Minute'] = data['Timestamp'].dt.minute
data

In [ ]:
data.drop(['Timestamp','Year','Month','Day','Day of Week'],axis=1, inplace=True)

# Rearrange order of columns
### Last column is the variable to be predicted

In [ ]:
prediction_column = data.columns.get_loc(prediction_variable) # 0-based
if (prediction_variable == 'MainIncoming_kW'):
    # Predicting for main variable; energy output
    data = data.iloc[:, np.r_[prediction_column + 1:len(data.columns), prediction_column]]
else:
    # Predicting for chiller variable
    data = data.iloc[:, np.r_[12:len(data.columns), prediction_column]]
data

# (Optional) Drop all weekend data

In [ ]:
weekend_condition = data['DayNumber'] >= 6
data_filtered = data.drop(data[weekend_condition].index)
data_filtered

# Preparing Training Data

In [ ]:
# Slice data into sequences
seq_len = 24*3*4 #3days in 15min interval
data_arr=[]
for i in range(len(data)-seq_len+1):
    data_slice = data[i:i+seq_len:]
    data_arr.append(data_slice)
data_arr = np.asarray(data_arr).astype('float32')
print(len(data_arr))

# Split Data

In [ ]:
# Split data
train = data_arr[:11444]
test_output = np.asarray(data)[11732:]
train_x, train_y = train[:,:-1,:-1], train[:,-1,-1]
test = data_arr[11732:]
test_x, test_y = test[:,:-1,:-1], test[:,-1,-1]
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)
# Construct the tensors, Reshape input to be 3D [samples, timesteps, features]
train_x = train_x.reshape((train_x.shape[0], seq_len-1, train_x.shape[2]))
test_x = test_x.reshape((test_x.shape[0], seq_len-1, test_x.shape[2]))
print(train_x)
print(test_x)

# (Optional) Find Best Hyperparameters

In [ ]:
def initLSTMModel(neurons, dropout_rate, activation):
    model = Sequential()
    model.add(LSTM(seq_len, input_shape=(train_x.shape[1], train_x.shape[2])))
    model.add(Dropout(dropout_rate))
    model.add(Dense(neurons, activation=activation))
    model.add(Dense(neurons, activation=activation))
    model.add(Dense(neurons, activation=activation))
    #model.add(Dense(1, activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(
        loss='mean_squared_error', 
        optimizer='adam', 
        metrics=[tf.keras.metrics.MeanSquaredError()]
        )
    return model

# Set up parameter grid
# Train with neurons, layers
param_grid = {
    'neurons': [24],

}

model = KerasRegressor(model=initLSTMModel, epochs=16, batch_size=32, neurons=24, dropout_rate=0.2, activation='relu')

# Perform grid search with cross-validation
# scoring='neg_mean_squared_error' as grid search will return maximum scoring
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error')
grid_result = grid_search.fit(train_x, train_y)

# Evaluate and extract the best model
best_model = grid_result.best_estimator_
test_accuracy = best_model.score(test_x, test_y)
print(best_model)
print(test_accuracy) 

# Create & Train Model 
### With best hyperparameters

In [ ]:
# Define the model
model = Sequential()
model.add(LSTM(seq_len, input_shape=(train_x.shape[1], train_x.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(24, activation='relu'))
model.add(Dense(24, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(
    loss='mean_squared_error', 
    optimizer='adam', 
    metrics='[tf.keras.metrics.MeanSquaredError()]
    )
# Train the model
history = model.fit(train_x, train_y, epochs=16, batch_size=32, validation_data=(test_x, test_y), verbose=True, shuffle=False)
# Plot training history
plt.plot(history.history['mean_absolute_error'], label='train')
plt.plot(history.history['val_mean_absolute_error'], label='test')
plt.legend()
plt.show()

# Save Model

In [ ]:
# Save the model
model.save('ml2023_model.h5')

In [ ]:
#This part is for testing purpose not really needed
past_10_day_test = pd.read_csv('Past_10_Days.csv')

# remove units from length column and convert to float
past_10_day_test['Timestamp'] = past_10_day_test['Timestamp'].str.replace(' Singapore', '').astype(str)
past_10_day_test['MainIncoming_kW'] = past_10_day_test['MainIncoming_kW'].str.replace('kW', '').astype(float)
past_10_day_test['CHWRT'] = past_10_day_test['CHWRT'].str.replace('°C', '').astype(float)
past_10_day_test['CWRT'] = past_10_day_test['CWRT'].str.replace('°C', '').astype(float)
past_10_day_test['CHWST'] = past_10_day_test['CHWST'].str.replace('°C', '').astype(float)
past_10_day_test['CWST'] = past_10_day_test['CWST'].str.replace('°C', '').astype(float)
past_10_day_test['CHW Flow'] = past_10_day_test['CHW Flow'].str.replace('L/s', '').astype(float)
past_10_day_test['CDW Flow'] = past_10_day_test['CDW Flow'].str.replace('L/s', '').astype(float)
past_10_day_test['heat_in'] = past_10_day_test['heat_in'].str.replace('_RT', '').astype(float)
past_10_day_test['heat_out'] = past_10_day_test['heat_out'].str.replace('_RT', '').astype(float)
past_10_day_test['lift'] = past_10_day_test['lift'].str.replace('Δ°C', '').astype(float)
past_10_day_test['chwdt'] = past_10_day_test['chwdt'].str.replace('Δ°C', '').astype(float)
past_10_day_test['cwdt'] = past_10_day_test['cwdt'].str.replace('Δ°C', '').astype(float)
past_10_day_test['DemoSite Demo avgHum'] = past_10_day_test['DemoSite Demo avgHum'].str.replace('%RH', '').astype(float)
past_10_day_test['DemoSite Demo avgTemp'] = past_10_day_test['DemoSite Demo avgTemp'].str.replace('°C', '').astype(float)
past_10_day_test['DemoSite Demo Calculated Wet Bulb'] = past_10_day_test['DemoSite Demo Calculated Wet Bulb'].str.replace('°C', '').astype(float)

actual_kW= past_10_day_test.loc[287:, ['MainIncoming_kW']]
print(actual_kW)

past_10_day_test['Timestamp'] =pd.to_datetime(past_10_day_test['Timestamp'],dayfirst=True)
# convert the timestamp to the corresponding day of the week
past_10_day_test['Day of Week'] = past_10_day_test['Timestamp'].dt.strftime('%A')
# Convert datetime to weekday numbers (Monday=1, Tuesday=2, etc.)
past_10_day_test['DayNumber'] = past_10_day_test['Timestamp'].dt.weekday
# add 1 to get days starting from 1
past_10_day_test['DayNumber'] = past_10_day_test['DayNumber'] + 1

past_10_day_test['Year'] = past_10_day_test['Timestamp'].dt.year
past_10_day_test['Month'] = past_10_day_test['Timestamp'].dt.month
past_10_day_test['Day'] = past_10_day_test['Timestamp'].dt.day
past_10_day_test['Hour'] = past_10_day_test['Timestamp'].dt.hour
past_10_day_test['Minute'] = past_10_day_test['Timestamp'].dt.minute
past_10_day_test.drop(['Timestamp','MainIncoming_kW','Year','Month','Day','Day of Week'],axis=1, inplace=True)
testing = past_10_day_test

data_arr1=[]
for i in range(len(past_10_day_test)-seq_len+1):
    data_slice = past_10_day_test[i:i+seq_len-1:]
    data_arr1.append(data_slice)
data_arr1 = np.asarray(data_arr1).astype('float32')
past_10_day_test = data_arr1.reshape((data_arr1.shape[0], seq_len-1, data_arr1.shape[2]))
test_Jan_Feb1 = model.predict(past_10_day_test)
test_Jan_Feb1 = pd.DataFrame(test_Jan_Feb1, columns=['Predicted'])
print(test_Jan_Feb1)

test_Jan_Feb1.to_csv("check_test.csv")
actual_kW.to_csv("check_actual.csv")
# Make predictions on test data
#test_pred = model.predict(test_x)

# Calculate MAE and MSE for test and predicted values
test_mae = mean_absolute_error(actual_kW, test_Jan_Feb1)
test_mse = mean_squared_error(actual_kW, test_Jan_Feb1)

print("Test MAE: ", test_mae)
print("Test MSE: ", test_mse)

# Define normalization function
def normalize(values):
    scaler = MinMaxScaler(feature_range=(0, 1))
    normalized = scaler.fit_transform(values.reshape(-1, 1))
    return normalized

#change df to array
test_Jan_Feb_final = test_Jan_Feb1.values
actual_kW = actual_kW.values

# Normalize test_y and predicted_y
actual_kW_norm = normalize(actual_kW)
predicted_kW_norm = normalize(test_Jan_Feb_final)

# Calculate normalized MAE and MSE
norm_mae = np.mean(np.abs(predicted_kW_norm - actual_kW_norm))
norm_mse = np.mean(np.square(predicted_kW_norm - actual_kW_norm))

print("Norm MAE: ", norm_mae)
print("Norm MSE: ", norm_mse)